In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader

In [2]:
model_local = ChatOllama(model="mistral")

In [3]:
loader = BSHTMLLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\mayo.html")

data = loader.load()
print(data)

[Document(page_content="\n\n      Addresses and phone numbers - About Us - Mayo Clinic\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThis content does not have an English version.\nThis content does not have an Arabic version.\n\n\n\nSkip to contentCare at \n   Mayo Clinic Patient-Centered Care About Mayo Clinic Request Appointment Find a Doctor Locations Clinical Trials Connect to Support Groups Patient & Visitor Guide Insurance & Billing Departments & Centers International Services Contact UsPatient & Visitor Guide Health \n   Library Diseases & Conditions Symptoms Tests & Procedures Drugs & Supplements Healthy Lifestyle Books & SubscriptionsDiseases & Conditions For Medical \n   Professionals Medical Professional Resources Refer a Patient Continuing Medical Education AskMayoExpert Mayo Clinic Laboratories Video Center Journals & Publications Mayo Clinic Alumni AssociationContinuing Medical Education Research & Educ

In [4]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(data)

In [5]:
# vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
# )
# retriever = vectorstore.as_retriever()

# Prepare your documents
# doc_splits =  # ... (Your list of text documents) 

# Instantiate the GPT4AllEmbeddings model
embeddings = GPT4AllEmbeddings()

# Create the Chroma vector store
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings
)

# Create the retriever
retriever = vectorstore.as_retriever()

In [6]:
print("Before RAG\n")
before_rag_template = "Provide me {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "the addresses of mayo clinic which is in Rochester"}))

Before RAG

 Mayo Clinic has several locations in Rochester, Minnesota. Here are some of the most common addresses:

1. Mayo Clinic Campus - 200 1st Street SW, Rochester, MN 55905, USA
This is the main campus and includes Mayo Clinic Hospital – Saint Marys Campus, Mayo Clinic Hospital – Methodist Campus, and Mayo Clinic's administrative offices.

2. Mayo Clinic Laboratories - 3434 West Frontage Drive NW, Rochester, MN 55901, USA
Mayo Clinic Laboratories provide diagnostic testing services for the entire Mayo Clinic system and beyond.

3. Gonda Building – 200 First Street SW, Rochester, MN 55904-0037, USA
The Gonda Building is home to Mayo Clinic's Dan Abraham Healthy Living Center, the Department of Physical Therapy and Rehabilitation, and several other clinics.

4. Mayo Clinic Health System – Methodist Building – 511 First Street SW, Rochester, MN 55902-0036, USA
This building houses Mayo Clinic's primary care clinic in downtown Rochester.

5. Mayo Clinic Health System – Saint Marys H

In [7]:
# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("provide the only address of Mayo clinic which is in Minnesota"))

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



########
After RAG

 The Mayo Clinic has multiple locations, but one of them is at Mayo Clinic, Rochester, Minnesota, with the following address:

Mayo Clinic
1216 Second St. SW,
Rochester, MN 55902, USA


Department of Radiology, Intervention Radiology section, Siriraj Hospital, Bangkok noi, Mahidol University, Thailand.
---

In [3]:
loader = BSHTMLLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\llm_testing_urls\\test_3.html")

data = loader.load()
print(data)

[Document(page_content='\n\n\n\n\n\n\n\n   Contact us - SIRIRAJ\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n       We value your privacy\n      \n\n\n\n         We use cookies to enhance your browsing experience, serve personalized ads or content, and analyze our traffic. By clicking "Accept All", you consent to our use of cookies.\n        \n\n\n\n         Customize\n        \n\n         Reject All\n        \n\n         Accept All\n        \n\n\n\n\n\n\n\n\n\n       Customize Consent Preferences\n      \n\n\n\n\n\n\n\n        We use cookies to help you navigate efficiently and perform certain functions. You will find detailed information about all cookies under each consent category below.\n       \n\n        The cookies that are categorized as "Necessary" are stored on your browser as they are essential for enabling the basic functionalities of the 

In [5]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(data)
print(len(doc_splits))

8


In [6]:
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

In [7]:
print("Before RAG\n")
before_rag_template = "Provide me {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "addresses and contact number and email address of The Siriraj Hospital"}))

Before RAG

 The Siriraj Hospital, also known as Siriraj Institute for Health Sciences, is located in Bangkok, Thailand. Here are its contact details:

Address:
Siriraj Hospital
2 Samuel Vajiravudh Avenue,
Bangkok 10200, Thailand

Contact Number:
General Inquiries: +66 2419 3500 or +66 2419 3700
Emergency Room: +66 2419 3555
International Patient Services: +66 2419 3551 or +66 2419 3552

Email Address:
For general inquiries and appointment scheduling, you can email at [sirirajhosp@mahidol.ac.th](mailto:sirirajhosp@mahidol.ac.th)
International Patient Services: [intlpatient@mahidol.ac.th](mailto:intlpatient@mahidol.ac.th)

Website:
You can also visit their website at https://www.siriraj.mahidol.ac.th/ for more information about the hospital's services, departments, and facilities.


In [15]:
# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("extract the all phone numbers from the given context"))


########
After RAG

 The context does not contain any explicit phone numbers mentioned. However, there are two phone numbers provided in the extracted documents from the context:

1. +66 2 419 7000 (Thai Language Only)
2. +66 2-419-1000 or domestic call 1474

These phone numbers can be extracted from the given documents.


In [16]:
after_rag_chain.invoke("extract the email address from the given context")

' siiro@mahidol.ac.th'